# Gemini API: Customer Support Agent

### Install Packages

In [ ]:
!pip install -U -q google-generativeai

### Import packages

Import the necessary packages.

In [ ]:
import pathlib
import textwrap
import time

import google.generativeai as genai
import google.ai.generativelanguage as glm

from IPython import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### Set up your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>


In [ ]:
try:
    # Used to securely store your API key
    from google.colab import userdata

    # Or use `os.getenv('API_KEY')` to fetch an environment variable.
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
except ImportError:
    import os
    GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

genai.configure(api_key=GOOGLE_API_KEY)

### Customer Support Chatbot

In [ ]:
def get_order_status(order_id: str) -> str:
    """Fetches the status of a given order ID."""
    # Mock data for example purposes
    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }
    return order_statuses.get(order_id, "Order ID not found.")

def initiate_return(order_id: str, reason: str) -> str:
    """Initiates a return for a given order ID with a specified reason."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Return initiated for order {order_id} due to: {reason}."
    else:
        return "Order ID not found. Cannot initiate return."

### Create Gemini Client

In [ ]:
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest',
    tools=[get_order_status, initiate_return] # list of all available tools
)

In [ ]:
model._tools.to_proto()

[function_declarations {
   name: "get_order_status"
   description: "Fetches the status of a given order ID."
   parameters {
     type_: OBJECT
     properties {
       key: "order_id"
       value {
         type_: STRING
       }
     }
     required: "order_id"
   }
 }
 function_declarations {
   name: "initiate_return"
   description: "Initiates a return for a given order ID with a specified reason."
   parameters {
     type_: OBJECT
     properties {
       key: "order_id"
       value {
         type_: STRING
       }
     }
     properties {
       key: "reason"
       value {
         type_: STRING
       }
     }
     required: "order_id"
     required: "reason"
   }
 }]

### alway use the model in chat mode for function calling

In [ ]:
chat = model.start_chat(enable_automatic_function_calling=True)

While this was all handled automatically, if you need more control, you can:

- Leave the default `enable_automatic_function_calling=False` and process the `glm.FunctionCall` responses yourself.
- Or use `GenerativeModel.generate_content`, where you also need to manage the chat history.

In [ ]:
response = chat.send_message('What is the status of order 12345?')
response.text

'The order is shipped. \n'

In [ ]:
response.candidates

[content {
  parts {
    text: "The order is shipped. \n"
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]

Examine the chat history to see the flow of the conversation and how function calls are integrated within it.

The `ChatSession.history` property stores a chronological record of the conversation between the user and the Gemini model. Each turn in the conversation is represented by a [`glm.Content`](https://ai.google.dev/api/python/google/ai/generativelanguage/Content) object, which contains the following information:

*   **Role**: Identifies whether the content originated from the "user" or the "model".
*   **Parts**: A list of [`glm.Part`](https://ai.google.dev/api/python/google/ai/generativelanguage/Part) objects that represent individual components of the message. With a text-only model, these parts can be:
    *   **Text**: Plain text messages.
    *   **Function Call** ([`glm.FunctionCall`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall)): A request from the model to execute a specific function with provided arguments.
    *   **Function Response** ([`glm.FunctionResponse`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse)): The result returned by the user after executing the requested function.

 In the previous example with the mittens calculation, the history shows the following sequence:

1.  **User**: Asks the question about the total number of mittens.
1.  **Model**: Determines that the multiply function is helpful and sends a FunctionCall request to the user.
1.  **User**: The `ChatSession` automatically executes the function (due to `enable_automatic_function_calling` being set) and sends back a `FunctionResponse` with the calculated result.
1.  **Model**: Uses the function's output to formulate the final answer and presents it as a text response.

In [ ]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What is the status of order 12345?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '12345'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Shipped'}}}
--------------------------------------------------------------------------------
model -> {'text': 'The order is shipped. \n'}
--------------------------------------------------------------------------------


In [ ]:
response = chat.send_message('I want to return order 11223 because it is defective.')
print(response.text)

OK. I have initiated a return for order 11223. 



In [ ]:
response.candidates

[content {
  parts {
    text: "OK. I have initiated a return for order 11223. \n"
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]

In [ ]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What is the status of order 12345?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '12345'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Shipped'}}}
--------------------------------------------------------------------------------
model -> {'text': 'The order is shipped. \n'}
--------------------------------------------------------------------------------
user -> {'text': 'I want to return order 11223 because it is defective.'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'initiate_return', 'args': {'reason': 'defective', 'order_id': '11223'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'initia

### Sequential Function Calls or Nested Calls

Output of the first function call becomes the input to the second!

    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }

In [ ]:
response = chat.send_message('Can you check the status of order 11223? If its delivered, please initiat return as it was the wrong order')
print(response.text)

Order 11223 has been delivered. I have initiated a return because it was the wrong order. 



In [ ]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What is the status of order 12345?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '12345'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Shipped'}}}
--------------------------------------------------------------------------------
model -> {'text': 'The order is shipped. \n'}
--------------------------------------------------------------------------------
user -> {'text': 'I want to return order 11223 because it is defective.'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'initiate_return', 'args': {'reason': 'defective', 'order_id': '11223'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'initia

# Sequential function calls - 2

    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }

In [ ]:
response = chat.send_message('Can you check the status of order 67890? If its delivered, please initiat return as it was the wrong order')
print(response.text)

Order 67890 is currently being processed. I can't initiate a return until it's delivered. 



In [ ]:
response = chat.send_message('Can you check the status of order 67890? If its delivered, please initiat return as it was the wrong order else cancel the order.')
print(response.text)

Order 67890 is currently being processed. I can't initiate a return until it's delivered.  I'll try to cancel it for you. 



In [ ]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What is the status of order 12345?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '12345'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Shipped'}}}
--------------------------------------------------------------------------------
model -> {'text': 'The order is shipped. \n'}
--------------------------------------------------------------------------------
user -> {'text': 'I want to return order 11223 because it is defective.'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'initiate_return', 'args': {'reason': 'defective', 'order_id': '11223'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'initia

### Adding a funciton to handle cancel

In [ ]:
def get_order_status(order_id: str) -> str:
    """Fetches the status of a given order ID."""
    # Mock data for example purposes
    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }
    return order_statuses.get(order_id, "Order ID not found.")

def initiate_return(order_id: str, reason: str) -> str:
    """Initiates a return for a given order ID with a specified reason."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Return initiated for order {order_id} due to: {reason}."
    else:
        return "Order ID not found. Cannot initiate return."

def cancel_order(order_id: str) -> str:
    """Cancels a given order ID if possible."""
    # Mock data for example purposes
    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }
    if order_id in order_statuses:
        if order_statuses[order_id] == "Processing":
            return f"Order {order_id} has been canceled successfully."
        else:
            return f"Order {order_id} cannot be canceled as it is already {order_statuses[order_id]}."
    else:
        return "Order ID not found. Cannot cancel order."

add cancel to the list of functions.

In [ ]:
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest',
    tools=[get_order_status, initiate_return, cancel_order]
)

In [ ]:
chat = model.start_chat(enable_automatic_function_calling=True)

In [ ]:
response = chat.send_message('Can you check the status of order 67890? If its delivered, please initiat return as it was the wrong order else cancel the order.')
print(response.text)

OK. I have canceled order 67890. 



In [ ]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'Can you check the status of order 67890? If its delivered, please initiat return as it was the wrong order else cancel the order.'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '67890'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Processing'}}}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'cancel_order', 'args': {'order_id': '67890'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'cancel_order', 'response': {'result': 'Order 67890 has been canceled successfully.'}}}
--------------------------------------------------------------------------------
model -> {'text': 'OK. I have canceled order 678

### Parallel Function Calling. Can it handle those?

    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }

In [ ]:
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest',
    tools=[get_order_status, initiate_return, cancel_order]
)

chat = model.start_chat(enable_automatic_function_calling=True)

In [ ]:
response = chat.send_message("What is the status of order 12345? Also, can you cancel order 67890 and initiate a return for order 11223 because it is defective?")

In [ ]:
response.text

'The status of order 12345 is Shipped. Order 67890 has been canceled successfully. A return has been initiated for order 11223 due to it being defective. \n'

### How many functions it can handle

In [ ]:
def get_order_status(order_id: str) -> str:
    """Fetches the status of a given order ID."""
    # Mock data for example purposes
    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }
    return order_statuses.get(order_id, "Order ID not found.")

def initiate_return(order_id: str, reason: str) -> str:
    """Initiates a return for a given order ID with a specified reason."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Return initiated for order {order_id} due to: {reason}."
    else:
        return "Order ID not found. Cannot initiate return."

def cancel_order(order_id: str) -> str:
    """Cancels a given order ID if possible."""
    # Mock data for example purposes
    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }
    if order_id in order_statuses:
        if order_statuses[order_id] == "Processing":
            return f"Order {order_id} has been canceled successfully."
        else:
            return f"Order {order_id} cannot be canceled as it is already {order_statuses[order_id]}."
    else:
        return "Order ID not found. Cannot cancel order."

def update_shipping_address(order_id: str, new_address: str) -> str:
    """Updates the shipping address for a given order ID."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Shipping address for order {order_id} has been updated to: {new_address}."
    else:
        return "Order ID not found. Cannot update shipping address."

def track_shipment(tracking_number: str) -> str:
    """Tracks the shipment with the given tracking number."""
    # Mock data for example purposes
    tracking_info = {
        "TRACK123": "In Transit",
        "TRACK456": "Delivered",
        "TRACK789": "Out for Delivery"
    }
    return tracking_info.get(tracking_number, "Tracking number not found.")

def apply_discount(order_id: str, discount_code: str) -> str:
    """Applies a discount to the given order ID."""
    # Mock data for example purposes
    valid_discount_codes = ["DISCOUNT10", "SAVE20"]
    if order_id in ["12345", "67890", "11223"]:
        if discount_code in valid_discount_codes:
            return f"Discount code {discount_code} applied to order {order_id}."
        else:
            return f"Invalid discount code: {discount_code}."
    else:
        return "Order ID not found. Cannot apply discount."

def change_payment_method(order_id: str, payment_method: str) -> str:
    """Changes the payment method for a given order ID."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Payment method for order {order_id} has been changed to: {payment_method}."
    else:
        return "Order ID not found. Cannot change payment method."

def provide_invoice(order_id: str) -> str:
    """Provides an invoice for the given order ID."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Invoice for order {order_id} has been sent to your email."
    else:
        return "Order ID not found. Cannot provide invoice."

def extend_warranty(order_id: str, years: int) -> str:
    """Extends the warranty for a given order ID."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Warranty for order {order_id} has been extended by {years} years."
    else:
        return "Order ID not found. Cannot extend warranty."

def check_product_availability(product_id: str) -> str:
    """Checks the availability of a product with the given product ID."""
    # Mock data for example purposes
    product_availability = {
        "PROD123": "In Stock",
        "PROD456": "Out of Stock",
        "PROD789": "Limited Stock"
    }
    return product_availability.get(product_id, "Product ID not found.")

In [ ]:
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest',
    tools=[
        get_order_status, initiate_return, cancel_order, update_shipping_address,
        track_shipment, apply_discount, change_payment_method, provide_invoice,
        extend_warranty, check_product_availability
    ]
)

In [ ]:
chat = model.start_chat()

In [ ]:
response = chat.send_message("What is the status of order 12345? Can you update the address to 123 Main St, Anytown USA?")
# response.text

In [ ]:
response.candidates

[content {
  parts {
    function_call {
      name: "get_order_status"
      args {
        fields {
          key: "order_id"
          value {
            string_value: "12345"
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "update_shipping_address"
      args {
        fields {
          key: "new_address"
          value {
            string_value: "123 Main St, Anytown USA"
          }
        }
        fields {
          key: "order_id"
          value {
            string_value: "12345"
          }
        }
      }
    }
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]

In [ ]:
# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

get_order_status(order_id=12345)
update_shipping_address(order_id=12345, new_address=123 Main St, Anytown USA)


In [ ]:
responses = {
    'get_order_status': get_order_status(order_id="12345"),
    'update_shipping_address': update_shipping_address(order_id="12345", new_address="123 Main St, Anytown USA")
}

In [ ]:
get_order_status(order_id="12345")

'Shipped'

In [ ]:
# Build the response parts.
response_parts = [
    glm.Part(function_response=glm.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

responses = chat.send_message(response_parts)
print(responses.text)

The order is currently shipped. I've updated the shipping address to 123 Main St, Anytown USA. 

